In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import cross_validate

In [ ]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if "startTime_for_tictoc" in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

In [ ]:
get_data = pd.read_csv('../input/heart-disease-prediction-using-logistic-regression/framingham.csv')
print(get_data.head(10))
print(get_data.isna().sum())

In [ ]:
Imputer = KNNImputer(n_neighbors=5)
Std = StandardScaler()
data_clean = pd.DataFrame(Imputer.fit_transform(get_data), columns=get_data.columns)
data_std = pd.DataFrame(Std.fit_transform(data_clean), columns=get_data.columns)
data_std['TenYearCHD'][data_std['TenYearCHD']<0]=0
data_std['TenYearCHD'][data_std['TenYearCHD']>0]=1
data_std.isna().sum()

In [ ]:
print(data_clean.head(10))
print(data_std.head(10))

In [ ]:
corr = data_std.corr()
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot()
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(get_data.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(get_data.columns)
ax.set_yticklabels(get_data.columns)
plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(data_std[get_data.columns[:-1]]), np.array(data_std[get_data.columns[15]]), test_size=0.2, shuffle=1)

# try K=1 through K=30 and record testing accuracy
k_range = list(range(1, 15))
scores = []

tic()
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    acc_pred = accuracy_score(y_test, y_pred)
    scores.append(acc_pred)
    # print(acc_pred)
toc()

# allow plots to appear within the notebook
%matplotlib inline
# plot the relationship between K and testing accuracy
plt.plot(k_range, scores)

print('max', np.argmax(scores, axis=0), max(scores))
print('min', np.argmin(scores, axis=0), min(scores))

knn = KNeighborsClassifier(n_neighbors=np.argmax(scores, axis=0)+1)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

hasil_conf = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(hasil_conf)
hasil_report = classification_report(y_test, y_pred)
print("Classification Report:",)
print (hasil_report)

# 10 Cross validation
cv = cross_validate(knn, x_test, y_test, return_train_score=True, cv=10, scoring=['accuracy'])
print(pd.DataFrame(cv))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(data_std[get_data.columns[:-1]]), np.array(data_std[get_data.columns[15]]), test_size=0.2, shuffle=1)

# try K=1 through K=30 and record testing accuracy
k_range = list(range(1, 30))
scores = []

tic()
for k in k_range:
    reg = KNeighborsRegressor(n_neighbors=k)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    error = np.sqrt(mean_squared_error(y_test,y_pred)) #calculate rmse
    scores.append(error) #store rmse values
    print('RMSE value for k= ' , k , 'is:', error)

In [ ]:
#plotting the rmse values against k values
curve = pd.DataFrame(scores) #elbow curve 
curve.plot()